In [1]:
import pandas as pd
import numpy as np

# Input

In [2]:
ICON = '../../experiments/exp10x/run1/cat/tables/abf5835_data_file_s1.csv'

In [3]:
TB = '../../experiments/exp10x/run1/cat/tables/tcr_barcode.csv'
TBC = '../../experiments/exp10x/run1/cat/tables/tcr_barcode.cleaned.csv' #
DF = '../../experiments/exp10x/run1/cat/eval_clonotypes/valid_ct.csv'

In [4]:
DONOR1 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor1_all_contig_annotations.csv'
CONTIG = '../../experiments/exp10x/run1/tcr/cellranger_tot/outs/multi/vdj_t/all_contig_annotations.csv'

In [5]:
CLEAN_TCR = '../../experiments/exp10x/run1/tcr/augmented/tcr.clean.augmented.csv'

# Output

In [189]:
ICON_OUT = '../../experiments/exp10x/run1/cat/eval_clonotypes/valid_ct.icon.csv'
WHLA_OUT= '../../experiments/exp10x/run1/cat/eval_clonotypes/valid_ct.hla.csv'

# Load

In [6]:
contig = pd.read_csv(CONTIG, low_memory=False)

In [7]:
icon_all = pd.read_csv(ICON,
                   usecols=['Cell barcode','Donor','pMHC','beta_v_gene','beta_j_gene','beta_cdr3','alpha_v_gene','alpha_j_gene','alpha_cdr3','UMI','corrected dex. Sig.','normalized dex. Sig.'],
                   encoding='unicode_escape')

In [13]:
df = pd.read_csv(DF, low_memory=False)

In [9]:
donor1 = pd.read_csv(DONOR1, low_memory=False)
tb = pd.read_csv(TB, low_memory=False)
tbc = pd.read_csv(TBC, low_memory=False)

In [10]:
tcr_clean = pd.read_csv(CLEAN_TCR, low_memory=False)

# Prep

In [11]:
gem_df = contig.barcode.str.split('-s', expand=True)
contig['gem'] = gem_df[0]
contig['gem_spl'] = gem_df[0].str.split('-', expand=True)[0]
contig['sample_id'] = 's' + gem_df[1]

### DF

In [12]:
gem_df = df.gem.str.split('-s', expand=True)
#df.gem = gem_df[0]
df['gem_spl'] = gem_df[0].str.split('-', expand=True)[0]
df.sample_id = 's' + gem_df[1]

In [13]:
sample2donor = {'s1':'Donor 1',
                's2':'Donor 2',
                's3':'Donor 3',
                's4':'Donor 4'}

In [14]:
df['donor'] = df.sample_id.map(sample2donor)

In [24]:
haplotypes = {'s1': ['A0201','A1101','B3501'],
              's2': ['A0201','A0101','B0801'],
              's3': ['A2402','A2902','B3502','B4403'],
              's4': ['A0301','B0702','B5701']}

In [16]:
df.HLA_cd8 = df.sample_id.map(haplotypes)

In [17]:
#df.to_csv(WHLA_OUT)

In [18]:
df.apply(lambda row: row.HLA_mhc in row.HLA_cd8, axis=1).sum()

70212

In [14]:
df.shape

(181913, 125)

In [16]:
df.gem.nunique()

181913

In [17]:
df.gem_spl.nunique()

161525

In [18]:
df.duplicated(subset=['gem','donor'], keep=False).sum()

0

In [19]:
df.duplicated(subset=['gem_spl','donor'], keep=False).sum()

11974

### ICON

In [20]:
icon_all.shape

(53062, 12)

In [43]:
icon_all[icon_all.Donor != 'Donor V'].shape

(39010, 12)

In [42]:
53062-14052

39010

In [21]:
# # Fix GEM IDs
# V_idx = icon[icon.Donor == 'Donor V'].index
# icon.loc[V_idx, 'Cell barcode'] = icon.loc[V_idx, 'Cell barcode'].str.replace('.1','-1', regex=False)

icon = icon_all[icon_all.Donor != 'Donor V'].copy()

In [22]:
gem_df = icon['Cell barcode'].str.split('.', expand=True)

icon['gem'] = gem_df[1]
icon['gem_spl'] = gem_df[1].str.split('-',expand=True)[0]
icon['sample_id'] = gem_df[0].str.replace('DV?','s', regex=True)

In [25]:
icon['HLA_cd8'] = icon.sample_id.map(haplotypes)

In [26]:
icon_pMHC = icon.pMHC.str.split('_', n=2, expand=True)
icon['peptide_HLA'] = icon_pMHC[1] +' '+ icon_pMHC[0].str.replace('*','', regex=False)
icon['HLA_mhc'] = icon_pMHC[0].str.replace('*','', regex=False)

In [27]:
icon.apply(lambda row: row.HLA_mhc in row.HLA_cd8, axis=1).sum()

32700

In [65]:
df.gem

0          AAACCTGAGAAACCGC-1-s3
1         AAACCTGAGAAACCTA-30-s2
2         AAACCTGAGAACTCGG-39-s2
3          AAACCTGAGAAGGGTA-9-s2
4          AAACCTGAGAATCTCC-3-s4
                   ...          
181908     TTTGTCATCTTAGAGC-2-s4
181909     TTTGTCATCTTGACGA-5-s3
181910    TTTGTCATCTTGAGAC-35-s2
181911    TTTGTCATCTTTAGGG-36-s2
181912     TTTGTCATCTTTAGGG-4-s3
Name: gem, Length: 181913, dtype: object

In [33]:
df.gem_spl

0         AAACCTGAGAAACCGC
1         AAACCTGAGAAACCTA
2         AAACCTGAGAACTCGG
3         AAACCTGAGAAGGGTA
4         AAACCTGAGAATCTCC
                ...       
181908    TTTGTCATCTTAGAGC
181909    TTTGTCATCTTGACGA
181910    TTTGTCATCTTGAGAC
181911    TTTGTCATCTTTAGGG
181912    TTTGTCATCTTTAGGG
Name: gem_spl, Length: 181913, dtype: object

In [34]:
df['va_match'] = df.v_gene_TRA == df.alpha_v_gene.str.replace('/','')
df['ja_match'] = df.j_gene_TRA == df.alpha_j_gene.str.replace('/','')
df['vb_match'] = df.v_gene_TRB == df.beta_v_gene.str.replace('/','')
df['jb_match'] = df.j_gene_TRB == df.beta_j_gene.str.replace('/','')

## Merge

In [50]:
tmp = pd.merge(df[df.columns[1:104]], icon, # added [df.columns[1:104]]
               left_on=['gem_spl','sample_id'],
               right_on=['gem_spl','sample_id'], # 270 gem_spl (all donor 2) do not exist in original data.
               how='left', suffixes=['','_icon'])

In [51]:
tmp['va_match'] = tmp.v_gene_TRA == tmp.alpha_v_gene.str.replace('/','')
tmp['ja_match'] = tmp.j_gene_TRA == tmp.alpha_j_gene.str.replace('/','')
tmp['vb_match'] = tmp.v_gene_TRB == tmp.beta_v_gene.str.replace('/','')
tmp['jb_match'] = tmp.j_gene_TRB == tmp.beta_j_gene.str.replace('/','')

In [72]:
tmp.loc[tmp.duplicated(subset=['Cell barcode'], keep=False) & ~tmp.gem_icon.isna(),
        ['gem','gem_icon','Cell barcode','clonotype','ct','peptide_HLA','peptide_HLA_icon','va_match','vb_match','ja_match','jb_match']].sort_values(by=['gem','Cell barcode','va_match','vb_match','ja_match','jb_match'])

,gem,gem_icon,Cell barcode,clonotype,ct,peptide_HLA,peptide_HLA_icon,va_match,vb_match,ja_match,jb_match
61,AAACCTGAGGCCCGTT-22-s2,AAACCTGAGGCCCGTT-17,D2.AAACCTGAGGCCCGTT-17,clonotype30394,11681,RAKFKQLL B0801,RAKFKQLL B0801,False,True,False,True
62,AAACCTGAGGCCCGTT-7-s2,AAACCTGAGGCCCGTT-17,D2.AAACCTGAGGCCCGTT-17,clonotype30344,1872,RAKFKQLL B0801,RAKFKQLL B0801,False,False,False,False
137,AAACCTGCACGCCAGT-22-s2,AAACCTGCACGCCAGT-16,D2.AAACCTGCACGCCAGT-16,clonotype31025,31025,IVTDFSVIK A1101,GILGFVFTL A0201,False,False,False,False
138,AAACCTGCACGCCAGT-28-s2,AAACCTGCACGCCAGT-16,D2.AAACCTGCACGCCAGT-16,clonotype30620,30620,GILGFVFTL A0201,GILGFVFTL A0201,True,True,True,True
165,AAACCTGCAGTATGCT-11-s2,AAACCTGCAGTATGCT-26,D2.AAACCTGCAGTATGCT-26,clonotype30546,30546,RAKFKQLL B0801,RAKFKQLL B0801,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
182743,TTTGTCACATGCCACG-7-s2,TTTGTCACATGCCACG-22,D2.TTTGTCACATGCCACG-22,clonotype30341,685,RAKFKQLL B0801,RAKFKQLL B0801,True,True,True,True
182850,TTTGTCATCAAAGTAG-27-s2,TTTGTCATCAAAGTAG-12,D2.TTTGTCATCAAAGTAG-12,clonotype31547,31547,GILGFVFTL A0201,RAKFKQLL B0801,False,False,False,False
182851,TTTGTCATCAAAGTAG-9-s2,TTTGTCATCAAAGTAG-12,D2.TTTGTCATCAAAGTAG-12,clonotype30537,30537,RAKFKQLL B0801,RAKFKQLL B0801,True,True,True,True
182864,TTTGTCATCAGTACGT-28-s2,TTTGTCATCAGTACGT-10,D2.TTTGTCATCAGTACGT-10,clonotype31589,31554,GILGFVFTL A0201,RAKFKQLL B0801,False,False,False,True


In [53]:
tmp['peptide_HLA_match'] = tmp.peptide_HLA == tmp.peptide_HLA_icon

OBS! There appears to be some mess in the ICON data regarding the GEM numbering. Alpha and beta chains are paired across different GEMs, e.g. AAACCTGTCTAACTTC-38 and AAACCTGTCTAACTTC-16. Moreover, the GEMs have a different numerical addition than the original data. Only 5177 GEM IDs are identical across ICON and 10x data.

In [83]:
tmp.sort_values(by=['gem','va_match','vb_match','ja_match','jb_match'], inplace=True)
# 2039 duplicates --> 1033 GEMs with had multiple entries in ICON
tmp.drop_duplicates(['gem','sample_id'], keep='first').dropna(subset=['gem_icon']).gem.nunique()#.to_csv(WHLA_OUT, index=False)

39813

In [62]:
tmp[~tmp.gem_icon.isna()]['Cell barcode'].nunique()

37654

In [184]:
tmp[~tmp.peptide_HLA_icon.isna()] #.to_csv(ICON_OUT, index=False)

39813

In [185]:
tmp.shape

(182946, 125)

In [161]:
icon[~icon.gem_spl.isin(contig.gem_spl)].shape

(270, 18)

In [180]:
icon[~icon.gem_spl.isin(df.gem_spl)].shape

(1133, 18)

In [181]:
icon.gem_spl.nunique()

38053

In [182]:
icon.shape

(39010, 18)

In [130]:
tmp.loc[tmp.duplicated(['gem','sample_id'], keep='first'), ['gem','gem_icon','sample_id','peptide_HLA','peptide_HLA_icon','genes_TRA','alpha_v_gene','genes_TRB','beta_v_gene']]

,gem,gem_icon,sample_id,peptide_HLA,peptide_HLA_icon,genes_TRA,alpha_v_gene,genes_TRB,beta_v_gene
289,AAACCTGTCTAACTTC-38,AAACCTGTCTAACTTC-34,s2,GILGFVFTL A0201,GILGFVFTL A0201,TRAV13-1;TRAJ42;TRAC,TRAV13-1,TRBV19;None;TRBJ2-3;TRBC2,TRBV19
290,AAACCTGTCTAACTTC-6,AAACCTGTCTAACTTC-13,s2,RAKFKQLL B0801,RAKFKQLL B0801,TRAV5;TRAJ6;TRAC,TRAV5,TRBV4-2;TRBD2;TRBJ2-2;TRBC2,TRBV4-2
385,AAACGGGAGGGATCTG-35,AAACGGGAGGGATCTG-40,s2,GILGFVFTL A0201,GILGFVFTL A0201,TRAV22;TRAJ37;TRAC,TRAV22,TRBV19;None;TRBJ2-7;TRBC2,TRBV19
386,AAACGGGAGGGATCTG-5,AAACGGGAGGGATCTG-1,s2,RAKFKQLL B0801,RAKFKQLL B0801,TRAV8-1;TRAJ21;TRAC,TRAV8-1,TRBV4-1;TRBD2;TRBJ2-1;TRBC2,TRBV4-1
1756,AAATGCCCAAGCGCTC-11,AAATGCCCAAGCGCTC-26,s2,GILGFVFTL A0201,GILGFVFTL A0201,TRAV8-6;TRAJ27;TRAC,TRAV8-6,TRBV6-3;TRBD1;TRBJ1-5;TRBC1,TRBV6-2
...,...,...,...,...,...,...,...,...,...
158488,TTTCCTCTCTCAAACG-20,TTTCCTCTCTCAAACG-23,s2,RAKFKQLL B0801,RAKFKQLL B0801,TRAV13-2;TRAJ45;TRAC,TRAV13-2,TRBV5-6;TRBD2;TRBJ1-1;TRBC1,TRBV5-6
158491,TTTCCTCTCTCAAACG-34,TTTCCTCTCTCAAACG-29,s2,RAKFKQLL B0801,RAKFKQLL B0801,TRAV1-1;TRAJ35;TRAC,TRAV1-1,TRBV27;None;TRBJ2-7;TRBC1,TRBV27
158493,TTTCCTCTCTCAAACG-40,TTTCCTCTCTCAAACG-29,s2,ELRRKMMYM B0801,RAKFKQLL B0801,TRAV21;TRAJ26;TRAC,TRAV1-1,TRBV5-8;TRBD1;TRBJ1-4;TRBC1,TRBV27
159238,TTTGTCAAGTGTTGAA-21,TTTGTCAAGTGTTGAA-8,s2,RAKFKQLL B0801,RAKFKQLL B0801,TRAV13-1;TRAJ50;TRAC,TRAV13-1,TRBV12-3;None;TRBJ2-2;TRBC2,TRBV12-3


In [121]:
contig.loc[contig.gem.str.contains('AAACCTGTCTAACTTC'), ['barcode','gem_spl','contig_id','length','chain','v_gene','d_gene','j_gene','cdr3']]

,barcode,gem_spl,contig_id,length,chain,v_gene,d_gene,j_gene,cdr3
187677,AAACCTGTCTAACTTC-38-s2,AAACCTGTCTAACTTC,AAACCTGTCTAACTTC-38_contig_1,546,TRA,TRAV13-1,None,TRAJ42,CAASMGGGSQGNLIF
187678,AAACCTGTCTAACTTC-38-s2,AAACCTGTCTAACTTC,AAACCTGTCTAACTTC-38_contig_2,853,TRB,TRBV19,None,TRBJ2-3,CASSIRSTDTQYF
187679,AAACCTGTCTAACTTC-38-s2,AAACCTGTCTAACTTC,AAACCTGTCTAACTTC-38_contig_3,444,TRB,None,None,TRBJ2-3,None
187680,AAACCTGTCTAACTTC-6-s2,AAACCTGTCTAACTTC,AAACCTGTCTAACTTC-6_contig_1,743,TRB,TRBV4-2,TRBD2,TRBJ2-2,CASSQRPSEVGELFF
187681,AAACCTGTCTAACTTC-6-s2,AAACCTGTCTAACTTC,AAACCTGTCTAACTTC-6_contig_2,824,TRB,TRBV4-2,TRBD2,TRBJ2-4,None
187682,AAACCTGTCTAACTTC-6-s2,AAACCTGTCTAACTTC,AAACCTGTCTAACTTC-6_contig_3,823,TRB,TRBV4-2,TRBD2,TRBJ1-4,None
187683,AAACCTGTCTAACTTC-6-s2,AAACCTGTCTAACTTC,AAACCTGTCTAACTTC-6_contig_4,499,TRA,TRAV5,None,TRAJ6,CAEMEGGSYIPTF
187684,AAACCTGTCTAACTTC-6-s2,AAACCTGTCTAACTTC,AAACCTGTCTAACTTC-6_contig_5,680,TRA,TRAV8-3,None,TRAJ10,None


# Compare

In [113]:
contig[(contig.is_cell == True) & (contig.productive == True) & (contig.full_length ==True)].nunique() #[contig.is_cell == True]

level_0                  4
level_1             230241
barcode             192765
is_cell                  1
contig_id           434735
high_confidence          2
length                1106
chain                    3
v_gene                 121
d_gene                   4
j_gene                  67
c_gene                  12
full_length              1
productive               1
cdr3                142027
cdr3_nt             153063
reads                46852
umis                    82
raw_clonotype_id     85354
raw_consensus_id     73356
gem_spl             169263
gem                 191454
sample_id                4
dtype: int64

In [105]:
df.shape

(181913, 103)

In [127]:
tmp.shape

(181913, 125)

In [123]:
tmp.dropna(subset=['gem_icon']).apply(lambda row: row.HLA_mhc_icon in row.HLA_cd8, axis=1).sum()

33808

In [136]:
tmp.to_csv(OUT, index=False)

In [129]:
tmp.gem

0          AAACCTGAGAAACCGC-1
1         AAACCTGAGAAACCTA-30
2         AAACCTGAGAACTCGG-39
3          AAACCTGAGAAGGGTA-9
4          AAACCTGAGAATCTCC-3
                 ...         
159687     TTTGTCATCTTAGAGC-2
159688     TTTGTCATCTTGACGA-5
159689    TTTGTCATCTTGAGAC-35
159690    TTTGTCATCTTTAGGG-36
159691     TTTGTCATCTTTAGGG-4
Name: gem, Length: 181913, dtype: object

In [132]:
contig.barcode.str.split('-s', expand=True)[0]

0          AAACCTGAGACAAAGG-4
1          AAACCTGAGACAAAGG-4
2          AAACCTGAGACAAAGG-4
3          AAACCTGAGACAAAGG-4
4          AAACCTGAGACAAAGG-4
                  ...        
1225327    TTTGTCATCTTGGGTA-5
1225328    TTTGTCATCTTTACGT-1
1225329    TTTGTCATCTTTACGT-1
1225330    TTTGTCATCTTTAGGG-5
1225331    TTTGTCATCTTTAGGG-5
Name: 0, Length: 1225332, dtype: object

/home/people/herpov/.conda/envs/snakemake/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (4,6,9,11,13,15,23,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [142]:
tcr_clean

,gem,clonotype,num_clonotype,ct,genes_TRA,genes_TRB,genes_lst_TRA,genes_lst_TRB,length_TRA,cdr3_TRA,...,v_gene_TRB,j_gene_TRB,tcr_category,no_filtration,exclude_single-chain_TCRs,exclude_ambiguous_and_single-chain_TCRs,exclude_ambiguous_TCRs,label,cell_flag,cell_high_confidence
0,AAACCTGAGAAACCGC-1-s3,clonotype50994,50994,50994,TRAV12-3;TRAJ39;TRAC,TRBV29-1;None;TRBJ1-2;TRBC1,['TRAV12-3;TRAJ39;TRAC'],['TRBV29-1;None;TRBJ1-2;TRBC1'],631.0,CAIGPGNMLTF,...,TRBV29-1,TRBJ1-2,unique chains,True,True,True,True,1.0,True,True
1,AAACCTGAGAAACCTA-30-s2,clonotype30341,30341,685,TRAV13-1;TRAJ50;TRAC,TRBV12-3;None;TRBJ2-2;TRBC2,['TRAV13-1;TRAJ50;TRAC'],['TRBV12-3;None;TRBJ2-2;TRBC2'],503.0,CAASETSYDKVIF,...,TRBV12-3,TRBJ2-2,unique chains,True,True,True,True,1.0,True,True
2,AAACCTGAGAAACGCC-2-s4,NaN,0,69110,TRAV21;TRAJ50;TRAC,NaN,['TRAV21;TRAJ50;TRAC'],NaN,522.0,CAVDLMKTSYDKVIF,...,NaN,NaN,missing chain,True,False,False,False,1.0,False,False
3,AAACCTGAGAACTCGG-39-s2,clonotype30489,30489,30489,TRAV2;TRAJ24;TRAC,TRBV6-6;TRBD2;TRBJ2-3;TRBC2,['TRAV2;TRAJ24;TRAC'],['TRBV6-6;TRBD2;TRBJ2-3;TRBC2'],458.0,CAVEGPTDSWGKFQF,...,TRBV6-6,TRBJ2-3,unique chains,True,True,True,True,1.0,True,True
4,AAACCTGAGAACTGTA-1-s3,NaN,0,51037,TRAV14DV4;TRAJ22;TRAC,NaN,['TRAV14DV4;TRAJ22;TRAC'],NaN,551.0,CAMRRPISSGSARQLTF,...,NaN,NaN,missing chain,True,False,False,False,1.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498249,TTTGTCATCTTCTGGC-9-s3,NaN,0,52182,NaN,TRBV12-4;TRBD2;TRBJ2-4;TRBC2,NaN,['TRBV12-4;TRBD2;TRBJ2-4;TRBC2'],NaN,NaN,...,TRBV12-4,TRBJ2-4,missing chain,True,False,False,False,1.0,False,False
498250,TTTGTCATCTTGCCGT-5-s4,NaN,0,70919,NaN,TRBV3-1;TRBD2;TRBJ2-3;TRBC2,NaN,['TRBV3-1;TRBD2;TRBJ2-3;TRBC2'],NaN,NaN,...,TRBV3-1,TRBJ2-3,missing chain,True,False,False,False,1.0,False,False
498251,TTTGTCATCTTGCCGT-7-s4,NaN,0,83732,NaN,TRBV19;None;TRBJ2-7;TRBC2,NaN,['TRBV19;None;TRBJ2-7;TRBC2'],NaN,NaN,...,TRBV19,TRBJ2-7,missing chain,True,False,False,False,1.0,False,False
498252,TTTGTCATCTTGGGTA-5-s4,NaN,0,120695,NaN,TRBV4-3;TRBD2;TRBJ2-1;TRBC2,NaN,['TRBV4-3;TRBD2;TRBJ2-1;TRBC2'],NaN,NaN,...,TRBV4-3,TRBJ2-1,missing chain,True,False,False,False,1.0,False,False


In [143]:
contig.barcode.nunique()

609037

In [148]:
contig.cdr3.str.contains('*', regex=False).sum()

9787

In [150]:
contig[~(contig.barcode.str.split('-s', expand=True)[0].isin(tmp.gem))].barcode.nunique()

423673

In [ ]:
icon[~icon.gem.str.split('-',expand=True)[0].isin(df_gem_split)]

,Cell barcode,Donor,pMHC,beta_v_gene,beta_j_gene,beta_cdr3,alpha_v_gene,alpha_j_gene,alpha_cdr3,UMI,corrected dex. Sig.,normalized dex. Sig.,gem
58,D1.AAAGCAAAGATTACCC-10,Donor 1,A*1101_IVTDFSVIK_EBNA 3B/EBV,TRBV11-2,TRBJ1-2,CASSWGGGSHYGYTF,TRAV35,TRAJ49,CAGHTGNQFYF,103,2.971037,2.992117,AAAGCAAAGATTACCC-10
84,D1.AAAGTAGGTGGCGAAT-38,Donor 1,A*0201_ELAGIGILTV_MART-1/Cancer/,TRBV19,TRBJ2-2,CASSLAGTAGELFF,TRAV12-2,TRAJ45,CAVGGGADGLTF,10,1.903789,14.526076,AAAGTAGGTGGCGAAT-38
90,D1.AAATGCCGTGAGGGAG-29,Donor 1,A*1101_IVTDFSVIK_EBNA 3B/EBV,TRBV5-1,TRBJ1-2,CASSLGQGVFYTF,TRAV35,TRAJ43,CAGLYNNNDMRF,359,2.463603,2.992117,AAATGCCGTGAGGGAG-29
139,D1.AACCGCGGTACGAAAT-12,Donor 1,A*0201_GILGFVFTL_Flu MP/Influenza,TRBV9,TRBJ2-3,CASSVDGDTSTDTQYF,TRAV3,TRAJ34,CAVRDPHQGTDKLIF,52,3.550150,4.085029,AACCGCGGTACGAAAT-12
141,D1.AACCGCGGTCATTAGC-15,Donor 1,A*1101_IVTDFSVIK_EBNA 3B/EBV,TRBV11-2,TRBJ1-2,CASSWGGGSHYGYTF,TRAV35,TRAJ49,CAGHTGNQFYF,231,3.526226,2.992117,AACCGCGGTCATTAGC-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53056,DV4.TTTGGTTTCCCATTAT-1,Donor V,A*0201_NLVPMVATV_pp65/CMV,TRBV12.4*01,TRBJ2.1*01,CASSMVNEQFF,TRAV3*01,TRAJ43*01,CAVRDTGDMRF,262,5.308616,1.348898,TTTGGTTTCCCATTAT-1
53058,DV4.TTTGTCACACATCTTT-1,Donor V,A*0201_NLVPMVATV_pp65/CMV,TRBV6.5*01,TRBJ1.2*01,CASSYVTGTGSYGYTF,TRAV23/DV6*01,TRAJ40*01,CAASPDSGTYKYIF,231,4.188268,1.348898,TTTGTCACACATCTTT-1
53059,DV4.TTTGTCACACCAGATT-1,Donor V,B*3501_IPSINVHHY_pp65/CMV,TRBV2*01,TRBJ2.2*01,CATGSQGLGTGELFF,TRAV41*01,TRAJ35*01,CAVLGGFGNVLHC,663,6.248285,1.125262,TTTGTCACACCAGATT-1
53060,DV4.TTTGTCAGTTTGTTTC-1,Donor V,B*3501_IPSINVHHY_pp65/CMV,TRBV2*01,TRBJ1.3*01,CASTPEGSGSGNTIYF,TRAV41*01,TRAJ35*01,CAVLGGFGNVLHC,325,5.749620,1.125262,TTTGTCAGTTTGTTTC-1


In [ ]:
icon[icon['Cell barcode'].str.contains('AAACCTGTCTGCGGCA')]

,Cell barcode,Donor,pMHC,beta_v_gene,beta_j_gene,beta_cdr3,alpha_v_gene,alpha_j_gene,alpha_cdr3,UMI,corrected dex. Sig.,normalized dex. Sig.,gem,gem_spl,sample_id
19,D1.AAACCTGTCTGCGGCA-1,Donor 1,A*0201_GILGFVFTL_Flu MP/Influenza,TRBV19,TRBJ2-2,CASSSRSTGELFF,TRAV25,TRAJ42,CAGNYGGSQGNLIF,52,1.267911,4.085029,AAACCTGTCTGCGGCA-1,AAACCTGTCTGCGGCA,s1


In [271]:
contig[contig.barcode.str.contains('AAACCTGTCTGCGGCA')]

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
317,AAACCTGTCTGCGGCA-5,True,AAACCTGTCTGCGGCA-5_contig_1,True,855,TRB,TRBV19,TRBD1,TRBJ2-2,TRBC2,True,True,CASSSRSTGELFF,TGTGCCAGTAGTTCGCGGTCAACCGGGGAGCTGTTTTTT,15465,12,clonotype1366,clonotype1366_consensus_1
318,AAACCTGTCTGCGGCA-5,True,AAACCTGTCTGCGGCA-5_contig_2,False,882,TRB,TRBV19,TRBD1,TRBJ2-2,TRBC2,True,True,CASSSRSTGELFF,TGTGCCAGTAGTTCGCGGTCAACCGGGGAGCTGTTTTTT,1489,1,clonotype1366,None
319,AAACCTGTCTGCGGCA-5,True,AAACCTGTCTGCGGCA-5_contig_3,False,694,TRB,TRBV6-3,TRBD1,TRBJ2-7,TRBC2,True,True,CASSSGQGTNYEQYF,TGTGCCAGCAGTTCTGGACAGGGGACAAACTACGAGCAGTACTTC,3264,1,clonotype1366,None
320,AAACCTGTCTGCGGCA-5,True,AAACCTGTCTGCGGCA-5_contig_4,False,414,TRB,None,None,TRBJ2-3,TRBC2,False,None,None,None,2312,1,clonotype1366,None
321,AAACCTGTCTGCGGCA-5,True,AAACCTGTCTGCGGCA-5_contig_5,False,505,TRB,None,None,TRBJ2-1,TRBC2,False,None,None,None,1627,1,clonotype1366,None
322,AAACCTGTCTGCGGCA-5,True,AAACCTGTCTGCGGCA-5_contig_6,False,693,TRB,TRBV3-1,TRBD1,TRBJ1-2,TRBC1,True,False,CASSHPSDRDYGYTF,TGTGCCAGCAGCCATCCTTCTGACAGGGACTATGGCTACACCTTC,2303,2,clonotype1366,None
323,AAACCTGTCTGCGGCA-5,True,AAACCTGTCTGCGGCA-5_contig_7,True,600,TRA,TRAV25,None,TRAJ42,TRAC,True,True,CAGNYGGSQGNLIF,TGTGCAGGGAATTATGGAGGAAGCCAAGGAAATCTCATCTTT,6928,3,clonotype1366,clonotype1366_consensus_2
324,AAACCTGTCTGCGGCA-5,True,AAACCTGTCTGCGGCA-5_contig_8,False,573,TRA,TRAV36DV7,None,TRAJ13,TRAC,True,None,None,None,2593,1,clonotype1366,None


In [272]:
df[df.gem.str.contains('AAACCTGTCTGCGGCA')]

,gem,clonotype,num_clonotype,ct,genes_TRA,genes_TRB,genes_lst_TRA,genes_lst_TRB,length_TRA,cdr3_TRA,...,ct_pep,sample_hla,pep_match,hla_match,ct_match,valid_ct,train_label,test_label,gem_spl,donor
306,AAACCTGTCTGCGGCA-5,clonotype1366,1366,1366,TRAV25;TRAJ42;TRAC,TRBV19;TRBD1;TRBJ2-2;TRBC2,['TRAV25;TRAJ42;TRAC'],['TRBV19;TRBD1;TRBJ2-2;TRBC2' 'TRBV6-3;TRBD1;T...,600.0,CAGNYGGSQGNLIF,...,NaN,NaN,NaN,NaN,NaN,False,NaN,False,AAACCTGTCTGCGGCA,Donor 1
